In [2]:
!pip install langid
!pip install langdetect

In [4]:
!unzip '/content/nit.zip'

Archive:  /content/nit.zip
   creating: nit/
  inflating: nit/2020-03-22_1089.txt  
  inflating: nit/2020-03-22_1182.txt  
  inflating: nit/2020-03-22_1188.txt  
  inflating: nit/2020-03-22_13.txt   
  inflating: nit/2020-03-22_1459.txt  
  inflating: nit/2020-03-22_1477.txt  
  inflating: nit/2020-03-22_1609.txt  
  inflating: nit/2020-03-22_1781.txt  
  inflating: nit/2020-03-22_1827.txt  
  inflating: nit/2020-03-22_1829.txt  
  inflating: nit/2020-03-22_2187.txt  
  inflating: nit/2020-03-22_2235.txt  
  inflating: nit/2020-03-22_2380.txt  
  inflating: nit/2020-03-22_2598.txt  
  inflating: nit/2020-03-22_2627.txt  
  inflating: nit/2020-03-22_3067.txt  
  inflating: nit/2020-03-22_3069.txt  
  inflating: nit/2020-03-22_314.txt  
  inflating: nit/2020-03-22_3186.txt  
  inflating: nit/2020-03-22_3302.txt  
  inflating: nit/2020-03-22_3392.txt  
  inflating: nit/2020-03-22_3665.txt  
  inflating: nit/2020-03-22_3673.txt  
  inflating: nit/2020-03-22_3768.txt  
  inflating: nit/2020

In [5]:
import string
import re
import os
import sys
import pathlib
from pathlib import Path
import nltk
import langid
import nltk, re, string, collections
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import xml.etree.ElementTree as ET
vectorizer = CountVectorizer()
puncs = string.punctuation
rem = stopwords.words() + list(puncs)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def create_xml(temp,sheet_name):
    root = ET.Element('root')
    root.text = '\n'    # newline before the celldata element

    #textdata = ET.SubElement(root, 'textdata')
    #textdata.text = '\n'    # newline before the collected element
    #textdata.tail = '\n\n'  # empty line after the textdata element
    for val1,val2,val3 in zip(temp['id'],temp['text'],temp['created_at']):
        # Fix some problems with the title as it will be used
        # as the tag name.
        textdata = ET.SubElement(root, 'textdata')
        textdata.text = '\n'    # newline before the collected element
        textdata.tail = '\n\n'  # empty line after the textdata element
        title = 'id'
        e = ET.SubElement(textdata, title.lower())
        e.text = val1
        e.tail = '\n'

        title = 'text'
        e = ET.SubElement(textdata, title.lower())
        e.text = val2
        e.tail = '\n'

        title = 'created_at'
        e = ET.SubElement(textdata, title.lower())
        e.text = val3
        e.tail = '\n'

    # Display for debugging            
    ET.dump(root)

    # Include the root element to the tree and write the tree
    # to the file.
    tree = ET.ElementTree(root)

    root_path = '/content/All_xml_files'

    completeName = os.path.join(root_path, sheet_name + ".xml")

    if not os.path.exists(root_path):
        os.makedirs(root_path)
    
    tree.write(completeName, encoding='utf-8', xml_declaration=True)

In [7]:
"""def make_xml_files(file_path):
    count = 1
    for path in pathlib.Path(file_path).iterdir():
        Sheet_name = str(path).split('/')[-1].split('.')[0]
        if path.is_file():
            current_file = open(path, "r")
            n_text = [line.split('},{') for line in current_file]
            temp = {'text':[],'created_at':[],'id':[]}
            for tex in n_text[0]:
                s = tex.split('","')
                if len(s) == 3:
                    orig_text = s[0].split('"text":"')[-1]
                    if langid.classify(orig_text)[0] == 'en':
                        text_tokens = word_tokenize(orig_text)
                        tokens_without_sw = [word for word in text_tokens if not word in rem]
                        filtered_sentence = (" ").join(tokens_without_sw)
                        temp['text'].append(filtered_sentence)
                        temp['created_at'].append(s[1].split('created_at":"')[-1])
                        temp['id'].append(s[2].split('id":"')[-1])
            create_xml(temp,Sheet_name)
            print('--------------stage-->' + str(count) + '--------------------' + str(Sheet_name) + '-------------')
            current_file.close()
        count += 1"""

'def make_xml_files(file_path):\n    count = 1\n    for path in pathlib.Path(file_path).iterdir():\n        Sheet_name = str(path).split(\'/\')[-1].split(\'.\')[0]\n        if path.is_file():\n            current_file = open(path, "r")\n            n_text = [line.split(\'},{\') for line in current_file]\n            temp = {\'text\':[],\'created_at\':[],\'id\':[]}\n            for tex in n_text[0]:\n                s = tex.split(\'","\')\n                if len(s) == 3:\n                    orig_text = s[0].split(\'"text":"\')[-1]\n                    if langid.classify(orig_text)[0] == \'en\':\n                        text_tokens = word_tokenize(orig_text)\n                        tokens_without_sw = [word for word in text_tokens if not word in rem]\n                        filtered_sentence = (" ").join(tokens_without_sw)\n                        temp[\'text\'].append(filtered_sentence)\n                        temp[\'created_at\'].append(s[1].split(\'created_at":"\')[-1])\n         

In [8]:
def make_xml_files(file_path):
    count = 1
    temp = {'text':[],'created_at':[],'id':[]}
    for path in pathlib.Path(file_path).iterdir():
        Sheet_name = str(path).split('/')[-1].split('.')[0]
        if path.is_file():
            current_file = open(path, "r")
            n_text = [line.split('},{') for line in current_file]
            for tex in n_text[0]:
                s = tex.split('","')
                if len(s) == 3:
                    orig_text = s[0].split('"text":"')[-1]
                    if langid.classify(orig_text)[0] == 'en':
                        text_tokens = word_tokenize(orig_text)
                        tokens_without_sw = [word for word in text_tokens if not word in rem]
                        filtered_sentence = (" ").join(tokens_without_sw)
                        temp['text'].append(filtered_sentence)
                        temp['created_at'].append(s[1].split('created_at":"')[-1])
                        temp['id'].append(s[2].split('id":"')[-1])
            print('--------------stage-->' + str(count) + '--------------------' + str(Sheet_name) + '-------------')
            current_file.close()
        count += 1
    create_xml(temp,'sample')
    return "Succesfully cretaed .xml file"

Give folder path to the function

In [ ]:
make_xml_files('/content/nit')

--------------stage-->1--------------------2020-05-03_1637-------------
--------------stage-->2--------------------2020-04-19_2243-------------
--------------stage-->3--------------------2020-06-10_2522-------------
--------------stage-->4--------------------2020-05-16_2405-------------
--------------stage-->5--------------------2020-06-05_2362-------------
--------------stage-->6--------------------2020-03-24_3940-------------
--------------stage-->7--------------------2020-04-18_1143-------------
--------------stage-->8--------------------2020-06-05_2656-------------
--------------stage-->9--------------------2020-05-04_161-------------
--------------stage-->10--------------------2020-05-05_3874-------------
--------------stage-->11--------------------2020-05-28_1525-------------
--------------stage-->12--------------------2020-05-10_2553-------------
--------------stage-->13--------------------2020-04-28_2658-------------
--------------stage-->14--------------------2020-05-24_2525--

In [ ]:
!cp -r "/content/All_xml_files" "drive/My Drive/GSP_Connectors"